In [15]:
import os
import pandas as pd
import prompts

from cohere import Client
from dotenv import load_dotenv
from tqdm import tqdm


In [16]:

load_dotenv()
co = Client(os.getenv("COHERE_API_KEY"))
model_name = "command-r-plus-08-2024"  # Latest release of Command-R Plus


In [17]:

filepath = "../datasets/perturbed_solutions_0.csv"
data = pd.read_csv(filepath)
data.head()


,id,question,solution,stepped,perturbed,step,type,trace
0,1,Given the functions $f(x) = \log_a(1+x)$ and $...,1. Since $f(x) = \log_a(1+x)$ and $g(x) = \log...,Step 1: Since $f(x) = \log_a(1+x)$ and $g(x) =...,Step 1: Since $f(x) = \log_a(1+x)$ and $g(x) =...,2,Dropped negative sign,"The second inequality should be $1-x > 0$, but..."
1,2,"In $\triangle ABC$, the lengths of the sides o...","Since $\cos \frac{C}{2} = \frac{\sqrt{5}}{3}$,...",Step 1: Since $\cos \frac{C}{2} = \frac{\sqrt{...,Step 1: Since $\cos \frac{C}{2} = \frac{\sqrt{...,2,Arithmetic sign error,The sign in the second term of the equation in...
2,3,Given that $P$ is any point on the circle $C$:...,The distance $d$ between the center of the cir...,Step 1: The distance $d$ between the center of...,Step 1: The distance $d$ between the center of...,2,Confusing variables,The variable $P$ was mistakenly used as $Q$ in...
3,4,Factorize: $x^{3}-6x^{2}+9x=\_\_\_\_\_\_.$,To factorize the given expression $x^{3}-6x^{2...,Step 1: To factorize the given expression $x^{...,Step 1: To factorize the given expression $x^{...,2,Algebraic manipulation error,The perturbation was applied by incorrectly fa...
4,5,Given that the sum of the first $n$ terms of a...,"Since $S\_n=2^{n}a\_n-1$,\nwe have $S_{n-1}=2^...","Step 1: Since $S\_n=2^{n}a\_n-1$, we have $S_{...","Step 1: Since $S\_n=2^{n}a\_n-1$, we have $S_{...",2,Arithmetic sign error,The subtraction operation in Step 2 was incorr...


In [18]:
def print_row_information(row: pd.Series):
    print("~~~Row Information~~~")
    print(f"ID: {row['id']}")
    print(f"Question: {row['question']}")
    print(f"Stepped Answer: {row['stepped']}")
    print(f"Type: {row['type']}")
    print(f"Step: {row['step']}")
    print(f"Trace: {row['trace']}")
    print(f"Perturbed: {row['perturbed']}")
    print("~~~End of Row Information~~~")


def get_row_completion(row: pd.Series):
    user_turn = prompts.COMPLETION_PROMPT_V2_USER.format(question=row["question"])
    assistant_turn = prompts.COMPLETION_PROMPT_V2_ASSISTANT.format(perturbed_reasoning=row["perturbed"])
    print(f"\n\n User propmt: {user_turn}\n\n Assistant prompt: {assistant_turn}\n\n")
    completion = co.chat(
        message=prompts.RAW_COMPLETION_TEMPLATE.format(user_turn=user_turn, assitant_turn=assistant_turn),
        raw_prompting=True,
    )
    print(f"\n\n Completion: {completion.text}\n\n")
    return completion.text


In [19]:
completion_df = data[0:10]

completions = []

for index, row in tqdm(completion_df.iterrows(), total=len(data), desc="Processing rows"):
    print("------------------Row------------------")
    print_row_information(row)
    completions.append(get_row_completion(row))
    print("------------------End of Row------------------")

completion_df["completion"] = completions

Processing rows:   0%|          | 0/50 [00:00<?, ?it/s]

------------------Row------------------
~~~Row Information~~~
ID: 1
Question: Given the functions $f(x) = \log_a(1+x)$ and $g(x) = \log_a(1-x)$, where $a>0$ and $a \neq 1$.
1. Find the domain of the function $f(x) - g(x)$.
2. Determine the parity of the function $f(x) - g(x)$.
3. Find the range of $x$ for which $f(x) - g(x) > 0$.
Stepped Answer: Step 1: Since $f(x) = \log_a(1+x)$ and $g(x) = \log_a(1-x)$, where $a>0$ and $a \neq 1$, we have $f(x) - g(x) = \log_a(1+x) - \log_a(1-x)$, where $a>0$ and $a \neq 1$. To ensure the function $f(x) - g(x)$ is meaningful, we need to consider the domain restrictions for both $\log_a(1+x)$ and $\log_a(1-x)$ to be defined.

Step 2: This gives us the system of inequalities:
$$
\begin{cases}
1+x > 0 \\
1-x > 0
\end{cases}
$$

Step 3: Solving this system, we find that $-1 < x < 1$, which indicates that the domain of the function $f(x) - g(x)$ is indeed $(-1, 1)$.

Step 4: Now, let's consider the symmetry of the function $f(x) - g(x)$ about the origin. 

Processing rows:   2%|▏         | 1/50 [00:09<07:57,  9.75s/it]



 Completion: 
Step 3: Solving the first inequality, we get $x > -1$. Solving the second inequality, we get $x < 1$. Therefore, the domain of $f(x) - g(x)$ is $\boxed{-1 < x < 1}$.

<answer>The domain of $f(x) - g(x)$ is $-1 < x < 1$.</answer>

Step 4: To determine the parity of the function $f(x) - g(x)$, we can examine the behavior of the function when $x$ is positive and when $x$ is negative.

Step 5: When $x$ is positive, $f(x) - g(x) = \log_a(1+x) - \log_a(1-x) > 0$ because $\log_a(1+x)$ is positive and $\log_a(1-x)$ is negative.

Step 6: When $x$ is negative, $f(x) - g(x) = \log_a(1+x) - \log_a(1-x) < 0$ because $\log_a(1+x)$ is negative and $\log_a(1-x)$ is positive.

Step 7: Therefore, the function $f(x) - g(x)$ is not even or odd.

<answer>The function $f(x) - g(x)$ is neither even nor odd.</answer>

Step 8: To find the range of $x$ for which $f(x) - g(x) > 0$, we can consider the behavior of the function within its domain.

Step 9: Within the domain $-1 < x < 1$, the functio

Processing rows:   4%|▍         | 2/50 [00:27<11:40, 14.60s/it]

...

Processing rows:  18%|█▊        | 9/50 [01:36<07:54, 11.58s/it]



 Completion: 
Step 8: **Solving for $a$ and $b$:**

Step 9: From the first equation, we have:

Step 10: $1\times a-b=1^{2}+2^{2}-3^{2}=1-5=-4$

Step 11: This gives us one equation for $a$ and $b$:

Step 12: $a-b=-4 \, .$

Step 13: Now, let's look at the second equation:

Step 14: $2\times 0-b=-11$

Step 15: So, we find the value of $b$:

Step 16: $b=11 \, .$

Step 17: Substituting $b=11$ into the equation $a-b=-4$, we can solve for $a$:

Step 18: $a-11=-4$

Step 19: $a=-4+11=7$

Step 20: Therefore, we have found the values:

Step 21: $a=7$ and $b=11$.

Step 22: #### Part 2: Conjecture for the $n$th Equation

Step 23: **Conjecture:**

Step 24: For the $n$th equation in the sequence, the pattern suggests that the equation will be of the form:

Step 25: $$(n-1)^2 + n^2 - (n+1)^2 = n \times (n-2) - 11$$

Step 26: **Proof:**

Step 27: We will prove this conjecture by mathematical induction on $n$.

Step 28: **Base Case ($n=1$):**

Step 29: For $n=1$, the equation becomes:

Step 30: $0^2 +

Processing rows:  20%|██        | 10/50 [01:40<06:41, 10.04s/it]



 Completion: 
Step 3: The sum of the exterior angles of any polygon is always $360^{\circ}$. So, we can set up the equation:

$$324^{\circ} \cdot \text{number of sides} = 360^{\circ}$$

Step 4: Solving for the number of sides, we get:

$$\text{number of sides} = \frac{360^{\circ}}{324^{\circ}} = \frac{5}{6} \cdot 6 = \boxed{5}$$

<answer>None of the above</answer>


------------------End of Row------------------



/tmp/ipykernel_1164289/211227130.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completion_df["completion"] = completions


In [24]:
completion_df.head()
# Completion_df columns include id, question, solution, stepped, perturbed, step, type, trace, completion
# All columns are int besides id and step
completion_df[completion_df["id"]==3].iloc[0]["stepped"]

'Step 1: The distance $d$ between the center of the circle $C(2,2)$ and the line $l$: $x+y=1$ is $d= \\frac{|2+2-1|}{ \\sqrt{2}}= \\frac{3}{ \\sqrt{2}} > 1$, hence the line $l$ and the circle $C$ are separate.\n\nStep 2: Let the coordinates of $P$ be $(x,y)$, then $P$ is any point on the circle $C$: $(x-2)^{2}+(y-2)^{2}=1$.\n\nStep 3: Let the coordinates of $Q$ be $(a,1-a)$, then $Q$ is any point on the line $l$: $x+y=1$.\n\nStep 4: Thus, $\\overrightarrow{OP}+ \\overrightarrow{OQ}=(x+a,y+1-a)$, and $| \\overrightarrow{OP}+ \\overrightarrow{OQ}|= \\sqrt{(x+a)^{2}+(y+1-a)^{2}}$, which represents the distance from the point $(-a,a-1)$ to any point on the circle $C$: $(x-2)^{2}+(y-2)^{2}=1$.\n\nStep 5: Let the distance between the point $(-a,a-1)$ and the center of the circle $C(2,2)$ be $d$, then the minimum value of $| \\overrightarrow{OP}+ \\overrightarrow{OQ}|$ is $d-1$.\n\nStep 6: We have $d= \\sqrt{(-a-2)^{2}+(a-1-2)^{2}}= \\sqrt{2a^{2}-2a+13}= \\sqrt{2(a- \\frac{1}{2})^{2}+ \\frac{

In [21]:
from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
completion_df.to_csv(f"../datasets/completions/solutions_perturbed_0_completions_command_r-{now}.csv", index=False)